### Task 1

Scrape weather information from an HTML file and process it using Python and BeautifulSoup.

<h4>5-Day Weather Forecast</h4>
<table>
    <thead>
        <tr>
            <th>Day</th>
            <th>Temperature</th>
            <th>Condition</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Monday</td>
            <td>25°C</td>
            <td>Sunny</td>
        </tr>
        <tr>
            <td>Tuesday</td>
            <td>22°C</td>
            <td>Cloudy</td>
        </tr>
        <tr>
            <td>Wednesday</td>
            <td>18°C</td>
            <td>Rainy</td>
        </tr>
        <tr>
            <td>Thursday</td>
            <td>20°C</td>
            <td>Partly Cloudy</td>
        </tr>
        <tr>
            <td>Friday</td>
            <td>30°C</td>
            <td>Sunny</td>
        </tr>
    </tbody>
</table>


Assume you are given the following HTML structure (you can save it as `weather.html`):

```html
<!DOCTYPE html>
<html>
<head>
    <title>Weather Forecast</title>
</head>
<body>
    <h4>5-Day Weather Forecast</h4>
    <table>
        <thead>
            <tr>
                <th>Day</th>
                <th>Temperature</th>
                <th>Condition</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Monday</td>
                <td>25°C</td>
                <td>Sunny</td>
            </tr>
            <tr>
                <td>Tuesday</td>
                <td>22°C</td>
                <td>Cloudy</td>
            </tr>
            <tr>
                <td>Wednesday</td>
                <td>18°C</td>
                <td>Rainy</td>
            </tr>
            <tr>
                <td>Thursday</td>
                <td>20°C</td>
                <td>Partly Cloudy</td>
            </tr>
            <tr>
                <td>Friday</td>
                <td>30°C</td>
                <td>Sunny</td>
            </tr>
        </tbody>
    </table>

</body>
</html>
```

1. **Parse the HTML File**:
   - Load the `weather.html` file using BeautifulSoup and extract the weather forecast details.

2. **Display Weather Data**:
   - Print the **day**, **temperature**, and **condition** for each entry in the forecast.

3. **Find Specific Data**:
   - Identify and print the day(s) with:
     - The highest temperature.
     - The "Sunny" condition.

4. **Calculate Average Temperature**:
   - Compute and print the **average temperature** for the week.



In [3]:
from bs4 import BeautifulSoup

# Load the HTML file
with open('weather.html', 'r') as file:
    soup = BeautifulSoup(file, 'html.parser')

# Extract weather forecast details
forecast = []
rows = soup.find('tbody').find_all('tr')
for row in rows:
    day = row.find_all('td')[0].text
    temperature = row.find_all('td')[1].text
    condition = row.find_all('td')[2].text
    forecast.append({'Day': day, 'Temperature': temperature, 'Condition': condition})

# Display weather data
for entry in forecast:
    print(f"Day: {entry['Day']}, Temperature: {entry['Temperature']}, Condition: {entry['Condition']}")

Day: Monday, Temperature: 25В°C, Condition: Sunny
Day: Tuesday, Temperature: 22В°C, Condition: Cloudy
Day: Wednesday, Temperature: 18В°C, Condition: Rainy
Day: Thursday, Temperature: 20В°C, Condition: Partly Cloudy
Day: Friday, Temperature: 30В°C, Condition: Sunny


### Task 2

Scrape job listings from the website https://realpython.github.io/fake-jobs and store the data into an SQLite database.

1. **Scraping Requirements**:
   - Extract the following details for each job listing:
     - **Job Title**
     - **Company Name**
     - **Location**
     - **Job Description**
     - **Application Link**

2. **Data Storage**:
   - Store the scraped data into an SQLite database in a table named `jobs`.

3. **Incremental Load**:
   - Ensure that your script performs **incremental loading**:
     - Scrape the webpage and add only **new job listings** to the database.
     - Avoid duplicating entries. Use `Job Title`, `Company Name`, and `Location` as unique identifiers for comparison.

4. **Update Tracking**:
   - Add functionality to detect if an existing job listing has been updated (e.g., description or application link changes) and update the database record accordingly.

5. **Filtering and Exporting**:
   - Allow filtering job listings by **location** or **company name**.
   - Write a function to export filtered results into a CSV file.



In [4]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv

def scrape_jobs(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    jobs = []
    job_elements = soup.find_all('div', class_='card-content')
    for job_element in job_elements:
        title = job_element.find('h2', class_='title').text.strip()
        company = job_element.find('h3', class_='company').text.strip()
        location = job_element.find('p', class_='location').text.strip()
        application_link = job_element.find('a')['href']
        jobs.append((title, company, location, application_link))
    return jobs

def store_jobs(jobs):
    conn = sqlite3.connect('jobs.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS jobs
                 (title TEXT, company TEXT, location TEXT, application_link TEXT)''')
    for job in jobs:
        c.execute('''INSERT OR IGNORE INTO jobs (title, company, location, application_link)
                     VALUES (?, ?, ?, ?)''', job)
    conn.commit()
    conn.close()

def filter_jobs(location=None, company=None):
    conn = sqlite3.connect('jobs.db')
    c = conn.cursor()
    query = 'SELECT * FROM jobs WHERE 1=1'
    params = []
    if location:
        query += ' AND location=?'
        params.append(location)
    if company:
        query += ' AND company=?'
        params.append(company)
    c.execute(query, params)
    jobs = c.fetchall()
    conn.close()
    return jobs

def export_jobs_to_csv(jobs, filename):
    with open(filename, 'w', newline='') as csvfile:
        job_writer = csv.writer(csvfile)
        job_writer.writerow(['Title', 'Company', 'Location', 'Application Link'])
        job_writer.writerows(jobs)

jobs = scrape_jobs("https://realpython.github.io/fake-jobs")
store_jobs(jobs)

filtered_jobs = filter_jobs(location='Remote')
export_jobs_to_csv(filtered_jobs, 'filtered_jobs.csv')

### Task 3

You are tasked with scraping laptop data from the "Laptops" section of the [Demoblaze website](https://www.demoblaze.com/) and storing the extracted information in JSON format.

**Steps:**

1. **Navigate to the Website:**
   - Visit the [Demoblaze homepage](https://www.demoblaze.com/).
   - Click on the **Laptops** section to view the list of available laptops.

2. **Navigate to the Next Page:**
   - After reaching the Laptops section, locate and click the **Next** button to navigate to the next page of laptop listings.

3. **Data to Scrape:**
   For each laptop on the page, scrape the following details:
   - **Laptop Name**
   - **Price**
   - **Description**

4. **Data Storage:**
   - Save the extracted information in a structured **JSON format** with fields like:
     ```json
     [
       {
         "name": "Laptop Name",
         "price": "Laptop Price",
         "description": "Laptop Description"
       },
       ...
     ]
     ```

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_laptops(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    laptops = []
    laptop_elements = soup.find_all('div', class_='card-block')
    for laptop_element in laptop_elements:
        name = laptop_element.find('h4', class_='card-title').text.strip()
        price = laptop_element.find('h5').text.strip()
        description = laptop_element.find('p', class_='card-text').text.strip()
        laptops.append({'name': name, 'price': price, 'description': description})
    return laptops

def scrape_all_laptops(base_url):
    all_laptops = []
    page = 1
    while True:
        url = f"{base_url}#page={page}"
        laptops = scrape_laptops(url)
        if not laptops:
            break
        all_laptops.extend(laptops)
        page += 1
    return all_laptops

base_url = "https://www.demoblaze.com/index.html"

all_laptops = scrape_all_laptops(base_url)

with open('laptops.json', 'w') as json_file:
    json.dump(all_laptops, json_file, indent=4)

print("Laptop data has been scraped and saved to laptops.json")

Laptop data has been scraped and saved to laptops.json
